# Working on Data Shape

In many programming languages, the programmer has to declare the dimensions of an array statically, and it is often possible to operate on the individual items using programmer-written loops.

In contrast to this, because APL processes arrays in their entirety, it is important to be able to manage the dimensions of an array dynamically.
This is why this chapter presents a number of new tools that will help you perform these tasks.

We have already studied functions which create arrays with specific shapes:

| Name | Syntax | Explanation |
| :- | :-: | :- |
| Reshape | `x⍴y` | Creates a new array with dimensions `x` and the data from `y`. |
| Catenate | `x,y` | Creates a new array by gluing `x` and `y` together. |
| Ravel | `,y` | Creates a vector from any array. |
| Compress | `x/y` | Selects parts of an array. |
| Replicate | `x/y` | Generally replicates the items of an array. |
| Indexing | `x[y]` | Creates a new array, often with modified dimensions. |
| Index function | `x⌷y` | Creates a new array, often with modified dimensions. |

## Take and Drop

### Take and Drop Applied to Vectors

#### Starter

There are two functions, _take_ (`↑`) and _drop_ (`↓`), that can be used to extract or remove a number of elements from the beginning or end of a vector.
The number is the left argument and its sign controls if the extraction/removal is done from the beginning or end of the vector.

 - _Take_ extracts the vector's head or tail; and
 - _Drop_ removes the vector's head or tail, and hence selects the remaining part.

Let us test these functions:

In [1]:
nums ← 56 66 19 37 44 20 18 23 68 70 82

With a positive left argument `n`, _take_ extracts the first `n` items:

In [2]:
4↑nums

56 66 19 37

It works on any kind of data (numbers, text),

In [3]:
5↑'My name is Bond'

My na

including nested arrays:

In [4]:
2↑(6 2)(35 33 26 21)(42 73)(1 2 3)

┌───┬───────────┐
│6 2│35 33 26 21│
└───┴───────────┘

With a negative left argument `n`, it extracts the last `|n` items of the vector, in their normal order:

In [5]:
¯3↑nums

68 70 82

In [6]:
¯6↑'Mississippi'

ssippi

Sometimes, these two operations are referred to as taking the "head" of the vector (with a positive left argument) and taking the "tail" of the vector (with a negative left argument).

With a positive left argument `n`, _drop_ removes the first `n` items and returns the tail:

In [7]:
4↓nums

44 20 18 23 68 70 82

In [8]:
5↓'My name is Bond'

me is Bond

With a negative left argument `n`, it removes the last `|n` items and returns the head:

In [9]:
¯6↓'Mississippi'

Missi

<!--begin remark-->
***Remark***:

 > You will have noticed that `4↑nums` and `¯7↓nums` both gave the same result `56 66 19 37`.
 >
 > At first sight it would appear that there is no need for both of these functions, and that one or other of _take_ and _drop_ is redundant.
 > There are, however, some differences that make it necessary to have both functions, as we will soon see.
<!--end-->

#### Be Careful

Do not confuse the two following expressions.

We can take the **last** three items of a vector:

In [10]:
¯3↑nums

68 70 82

Or we can take the first three items, and then change their sign:

In [11]:
-3↑nums

¯56 ¯66 ¯19

On another note, the result of _take_ and _drop_ applied to a vector remains a vector, even if it has only one item:

In [12]:
⍴1↑nums

1

In [13]:
⍴10↓nums

1

Although the results of the two expressions above would contain only one item, they are 1-item vectors, and not scalars.

#### Produce Empty Vectors

Of course, if you take no items, or if you drop all the items, the result is an empty vector of the same type (numeric or character) as the original vector:

In [14]:
0↑nums

In [15]:
22↓nums

In [16]:
14↓'Empty'

Although all of the results above are empty vectors, we know that they are different if the original vectors have different types of data:

In [17]:
(22↓nums)≡(14↓'Empty')

0

Take a look at [this section](./Data-and-Variables.ipynb#Empty-Arrays) for a reminder of how empty arrays work.

#### Take More Cash Than You Have

The _take_ function has a very special property: it allows you to take more items than there really are.
If so, it pads the result with _fill items_; zeroes for a numeric vector, and blanks for a text vector:

In [18]:
cash ← 45 23 18 92

If we take too many items, zeroes are appended to the vector:

In [19]:
7↑cash

45 23 18 92 0 0 0

With negative left arguments, starting from the tail, the zeroes are placed before the existing items:

In [20]:
¯9↑cash

0 0 0 0 0 45 23 18 92

For character vectors, spaces are appended:

In [21]:
12↑'Invisible'

Invisible

However, you have to be careful because those do not show:

In [22]:
(12↑'Invisible'),'.'

Invisible .

Starting from the right, with a negative left argument, the blanks are appended to the left and are a bit easier to see:

In [23]:
¯12↑'Visible'

Visible

In fact, the concept of _fill item_ is a bit more complex than this; it will be studied in detail in [a later section](./Nested-Arrays-(Continued).ipynb#Prototype-Fill-Item).

The concept of "taking more than you have" is sometimes referred to as _overtaking_.
This is an application of _take_ that cannot be performed using _drop_ alone.

This property applies equally to empty vectors; they are filled with as many zeroes or blanks as specified.
This means that the result will be different for empty numeric and empty character vectors:

In [24]:
]display 4↑⍬

┌→──────┐
│0 0 0 0│
└~──────┘

In [25]:
]display 4↑''

┌→───┐
│ │
└────┘

### Three Basic Applications

#### Determine the Type of a Variable

The property we have just seen can be used to determine whether an array is numeric or character, provided that it is simple and homogeneous (neither _mixed_ nor _nested_).
The method is simple: create an empty vector "filled" with the array (`0⍴⍵`), then take one item of it (`1↑`), and compare with `0`.
This will return `1` (true) for a numeric array, and `0` (false) for a character array.
A little dfn will do that for us:

In [26]:
TypeOf ← {0=1↑0⍴⍵}

In [27]:
TypeOf nums

1

In [28]:
TypeOf 'Character vector'

0

This function wouldn't work on a mixed or nested variable.
Again, you will understand why when we talk about _fill items_ in [this section](./Nested-Arrays-(Continued).ipynb#Prototype-Fill-Item).
We shall see in [a later chapter](./System-Interfaces.ipynb) that APL has a _system function_ that does the job much better.

#### Change a Vector into a Matrix

Sometimes you want a variable `var` to be a matrix, although you are not sure of its current rank.

If it is already a matrix, you want to leave it unchanged, and if it is a vector, you want to change it into a one-row matrix.
The following function should help:

In [29]:
HorMat ← {(¯2↑1,⍴⍵)⍴⍵}

Notice that the shape of the following matrix remains unchanged:

In [30]:
⍴HorMat 3 5⍴⍳5

3 5

But the vector is "promoted" into a matrix:

In [31]:
⍴HorMat 'This is a vector'

1 16

Let's break down how `HorMat` works:

 1. we first append `1` to the shape of the argument, which gives `1 3 5` for the matrix and `1 16` for the vector;
 2. next, we keep only the last two items, which gives `3 5` for the matrix and `1 16` for the vector;
 3. the final _reshape_ returns an unchanged matrix, or transforms a vector intro a matrix.

We can change vectors into 1-column matrices with a very similar function, which also leaves matrices unchanged:

In [32]:
VerMat ← {(2↑(⍴⍵),1)⍴⍵}

In [33]:
VerMat 'Vertically?'

V
e
r
t
i
c
a
l
l
y
?

#### Calculate Growth Rates

Let us imagine a business with a turnover which has grown over 12 years.
The variable `tome` is TurnOver in Millions of Euros:

In [34]:
tome ← 56 59 67 64 60 61 68 73 78 75 81 84

We want to calculate the difference between each year and the next; how do we achieve this?

In [35]:
1↓tome

59 67 64 60 61 68 73 78 75 81 84

In [36]:
¯1↓tome

56 59 67 64 60 61 68 73 78 75 81

All that remains is to subtract the results of these expressions one from the other, item by item:

In [37]:
(1↓tome)-(¯1↓tome)

3 8 ¯3 ¯4 1 7 5 5 ¯3 6 3

If instead of subtraction we used division, we would calculate (with some adjustments) the rates of growth instead of the differences.
Let us upt that in a small dfn, and apply it:

In [38]:
Growth ← {100×((1↓⍵)÷(¯1↓⍵))-1}

In [39]:
2⍕Growth tome

5.36 13.56 ¯4.48 ¯6.25 1.67 11.48 7.35 6.85 ¯3.85 8.00 3.70

### Take and Drop Applied to Arrays

The functions _take_ and _drop_ can be applied to any array so long as the left argument contains a number of items that is equal to or smaller than the number of dimensions (the _rank_) of the array.

In the expressions `ns↑array` and `ns↓array`, `≢ns` must be less than or equal to `≢⍴array`.

For the examples that follow, we shall work on the following matrix:

In [40]:
mat ← 3 4⍴13 52 33 81 42 62 70 47 51 73 28 19

`mat` is an integer matrix but _take_ and _drop_ work the same for character arrays, and even for _nested_ and/or _mixed_ arrays.

#### Maximal-Length Left Argument

We start by exploring what happens when the left argument has the maximal length it can have, that is, when `(≢ns)=≢⍴array` in the expression `ns↑array` or `ns↓array`.

If we want to take the 2 top rows and the 3 left columns of `mat`, we write

In [41]:
2 3↑mat

13 52 33
42 62 70

The elements of the left argument are paired up with the axes of the right argument array, and that is why `2 3↑mat` takes 2 _rows_ and then 3 _columns_, because the first axis specifies the number of rows and the second one the number of columns.

In order to drop the top row and 2 columns _from the right_, we write

In [42]:
1 ¯2↓mat

42 62
51 73

If we take 5 rows starting from the bottom and 3 columns starting from the left, we get

In [43]:
¯5 3↑mat

0 0 0
 0 0 0
13 52 33
42 62 70
51 73 28

The two extra rows are added on the top of `mat` because we asked for 5 rows _starting from the bottom_.

Dropping 1 row starting from the bottom and 3 columns starting from the left, we get

In [44]:
¯1 3↓mat

81
47

In the example above, we have dropped 3 out of the 4 columns available, so there is only one left, but it is still a 1-column matrix; it has not been changed into a vector.
This is similar to how `1↑vector` in a vector returns a 1-element vector, not a scalar.
In short, the functions _take_ and _drop_ return a result with the same rank as the right argument, regardless of the number of elements in the result.

As for vectors, it is often possible to use _take_ or _drop_ interchangeably to obtain the same result:

In [45]:
2 ¯3↑mat

52 33 81
62 70 47

In [46]:
¯1 1↓mat

52 33 81
62 70 47

#### Short Left Argument

<!-- (TODO) Write about Leading Axis Theory -->
If one or more of the trailing dimensions of the array are to remain unchanged, then one need only specify the parameters for the leading dimensions.

For example, suppose that we want to extract the first 2 rows of `mat`.
We know its shape is `3 4`, so we can easily write

In [47]:
2 4↑mat

13 52 33 81
42 62 70 47

But if we didn't know its shape in advance, we would have to resort to something like

In [48]:
(2,¯1↑⍴mat)↑mat

13 52 33 81
42 62 70 47

which is a bit cumbersome and, as it turns out, far from optimal.

In a matrix, the first axis denotes the rows, so we can pick the first 2 rows by just specifying the `2` in the left argument to _take_:

In [49]:
2↑mat

13 52 33 81
42 62 70 47

This works for arrays of arbitrarily high rank.
For example, recall the 3D array `prod`, that contained monthly (12) production values of 2 assembly lines, throughout the course of 5 years:

In [58]:
⎕RL ← 73
⎕← prod ← ?5 2 12⍴50

9 16 42 50 2 3 11 45 17 37 29 36
34 19 32 12 37 15 46 24 49 28 36 9
 
29 5 45 23 27 4 23 16 39 22 22 2
46 20 47 17 18 25 13 38 42 38 45 28
 
37 19 5 10 30 49 16 18 46 47 47 3
39 23 41 20 6 40 21 22 40 49 20 1
 
13 36 12 40 12 15 24 20 21 12 19 25
 8 48 22 37 33 1 39 30 50 50 8 35
 
38 31 24 40 23 7 20 34 35 6 19 27
41 1 14 50 17 42 6 35 12 48 30 29

We can get the production values for the first 2 years with:

In [61]:
2↑prod

9 16 42 50 2 3 11 45 17 37 29 36
34 19 32 12 37 15 46 24 49 28 36 9
 
29 5 45 23 27 4 23 16 39 22 22 2
46 20 47 17 18 25 13 38 42 38 45 28

But we can also get the production values for the first 4 years and only for the first assembly line:

In [62]:
4 1↑prod

9 16 42 50 2 3 11 45 17 37 29 36
 
29 5 45 23 27 4 23 16 39 22 22 2
 
37 19 5 10 30 49 16 18 46 47 47 3
 
13 36 12 40 12 15 24 20 21 12 19 25

Naturally, _drop_ can also be used with a left argument whose length is smaller than the rank of the argument.
For example, we can drop the first row of a matrix:

In [69]:
1↓mat

42 62 70 47
51 73 28 19

Or we can drop the production values of the _last_ year and of the first assembly line:

In [70]:
¯1 1↓prod

34 19 32 12 37 15 46 24 49 28 36 9
 
46 20 47 17 18 25 13 38 42 38 45 28
 
39 23 41 20 6 40 21 22 40 49 20 1
 
 8 48 22 37 33 1 39 30 50 50 8 35

In [the exercises](#Exercises) you can find a rule to compute the shape of the final result of a _take_ (or _drop_) operation in terms of the left argument and the shape of the right argument.

#### Short Left Argument with Axis Specification

When you provide a left argument that has less elements than the number of axes on the right argument, the left argument defaults to referring to the leading axes of the right argument.
However, we can use _take_ and _drop_ with _axis_ `[]` to specify other axes.

For example, the first 2 rows of `mat` were obtained with

In [55]:
2↑mat

13 52 33 81
42 62 70 47

Using _axis_, we can just as easily get the first 2 columns of `mat`:

In [56]:
2↑[2]mat

13 52
42 62
51 73

This also extends to higher rank.
If we want the information regarding the first 3 years, and only the final 6 months of the year (but for all assembly lines), we can write:

In [71]:
3 ¯6↑[1 3]prod

11 45 17 37 29 36
46 24 49 28 36 9
 
23 16 39 22 22 2
13 38 42 38 45 28
 
16 18 46 47 47 3
21 22 40 49 20 1

## Laminate

We have previously used _catenate_ to glue one array to another; let us now look at a new method.

We shall work with the following two character matrices:

In [73]:
⎕← names ← 3 4⍴'AlexAnnaMark'

Alex
Anna
Mark

In [74]:
⎕← surnames ← 3 4⍴'GainNairTyte'

Gain
Nair
Tyte

The _catenation_ of those two matrices will not change the _rank_ of the arrays, and thus give another matrix, as we saw in [a previous chapter](./Some-Primitive-Functions.ipynb#Concatenate):

In [75]:
names,surnames

AlexGain
AnnaNair
MarkTyte

In [76]:
names⍪surnames

Alex
Anna
Mark
Gain
Nair
Tyte

If both matrices have exactly the same shape, it is possible to join them together along a new dimension to make a 3D array.
Because this operation produces a result of higher rank than its arguments, it is called _laminate_ rather than _catenate_.

The symbol representing _catenate_ and _laminate_ is the same (`,`), but when the comma is used as _laminate_ it is always used with a fractional _axis_.

The two arrays we intend to _laminate_ have the same shape: `3 4`.
Because we are going to laminate 2 arrays, the new dimension will have a length of 2, and the shape of the result will be some combination of `3`, `4`, and `2`.
Let us examine all the possibilities:

 - <code>2 3   4&nbsp;&nbsp;</code> – the new dimension is inserted before the first dimension;
 - <code>&nbsp;&nbsp;3 2 4&nbsp;&nbsp;</code> – the new dimension is inserted between the first and second dimensions;
 - <code>&nbsp;&nbsp;3   4 2</code> – the new dimension is inserted after the second dimension.

To obtain these three different results, we shall use _laminate_ with a fractional axis to specify where the new dimension is to be inserted:

 - `names,[0.5]surnames` will produce a result of shape <code>2 3   4&nbsp;&nbsp;</code>;
 - `names,[1.5]surnames` will produce a result of shape <code>&nbsp;&nbsp;3 2 4&nbsp;&nbsp;</code>; and
 - `names,[2.5]surnames` will produce a result of shape <code>&nbsp;&nbsp;3   4 2</code>.

Beware that we could have equally used `⍪`.

Here are the three cases:

In [77]:
names,[0.5]surnames

Alex
Anna
Mark
 
Gain
Nair
Tyte

In [78]:
names,[1.5]surnames

Alex
Gain
 
Anna
Nair
 
Mark
Tyte

In [79]:
names,[2.5]surnames

AG
la
ei
xn
 
AN
na
ni
ar
 
MT
ay
rt
ke

In fact, the value of the axis specifier just identifies the _position_ of the new dimension relative to the values 1 and 2, so it could be any other fractional value between 0 and 1, or 1 and 2, or 2 and 3, respectively.

Hence, the three matches that we see below:

In [80]:
(names,[0.295]surnames)≡names,[0.5]surnames

1

In [81]:
(names,[1.643]surnames)≡names,[1.5]surnames

1

In [82]:
(names,[2.107]surnames)≡names,[2.5]surnames

1

Of course, it would be somewhat obtuse to use such axis specifications, and programmers conventionally use "`n.5`" values, like the ones in our examples.

### Applications to Vectors and Scalars

Now that we understand the reason for the fractional axis, which is perhaps initially somewhat surprising, we can apply _laminate_ to all kind of arrays.

#### Laminate Applied to Vectors

Let us use both character and numeric vectors:

In [92]:
t1 ← 'tomatoes'
t2 ← 'potatoes'
n1 ← 14 62 32 88
n2 ← 10×⍳4

If we catenate them, we still obtain vectors:

In [93]:
t1,t2

tomatoespotatoes

In [94]:
n1,n2

14 62 32 88 10 20 30 40

But if we instead laminate them, we obtain matrices with either 2 rows or 2 columns:

In [95]:
t1,[0.5]t2

tomatoes
potatoes

In [96]:
t1,[1.5]t2

tp
oo
mt
aa
tt
oo
ee
ss

In [97]:
n1,[0.5]n2

14 62 32 88
10 20 30 40

In [98]:
n1,[1.5]n2

14 10
62 20
32 30
88 40

Of course, since we are working with 1-dimensional arrays, we cannot specify an axis equal to or greater than 2.

There is also no reason for why you wouldn't be able to laminate a character vector with a numeric vector, for example:

In [99]:
t1,[0.5]n1,n2

t o m a t o e s
14 62 32 88 10 20 30 40

#### Laminate Scalars with Vectors

Scalars can be laminated with any array: they are repeated as many times as necessary to match the length of the new dimension:

In [101]:
n1,[0.5]0

14 62 32 88
 0 0 0 0

In [102]:
1,[1.5]n1

1 14
1 62
1 32
1 88

This can be used, for example, to underline a title:

In [103]:
title ← 'Laminate is good for you'

Without _laminate_, we must create a matrix with 2 rows, and as many columns as the length of `title`, filled with `title` itself, followed by as many dashes as the length of `title`: boring!

In [104]:
(2,≢title)⍴title,(≢title)⍴'-'

Laminate is good for you
------------------------

Now, with _laminate_, we just have to laminate a single dash; it will be repeated as many times as necessary:

In [105]:
title,[0.5]'-'

Laminate is good for you
------------------------

### Applications

#### Interlace Matrices

Do you remember that, in [the chapter about user-defined functions](./User-Defined-Functions.ipynb), we wrote a function to interlace two matrices?
It is no longer relevant; we can solve the problem more simply using _laminate_.

Can you see how? Give it some thought, if you will.

Take a look at the result of `names,[2.5]surnames` above.
You will see that the names are on the left and the surnames are on the right.
If we reshape that result with appropriate dimensions, we shall obtain `names` and `surnames` interlaced 😊 :

In [106]:
(1 2×⍴names)⍴names,[2.5]surnames

AGlaeixn
ANnaniar
MTayrtke

The result isn't easy to read, but the expression works!

Hence, our new implementation of `Interlace` could be

In [107]:
Interlace ← {(1 2×⍴⍺)⍴⍺,[2.5]⍵}

We can apply the same technique to matrices of forecasts and actuals, provided we define them first:

In [110]:
⎕RL ← 73
forecast ← 10×?4 6⍴55
⎕RL ← 73
actual ← forecast + ¯10+?4 6⍴20

In [111]:
forecast Interlace actual

90 89 160 166 420 420 500 508 20 12 30 23
110 111 450 453 170 177 370 365 290 284 360 352
340 349 190 192 320 329 120 115 510 515 370 374
150 160 460 467 240 234 520 519 490 485 280 283

#### Show Vectors

Sometimes you will have two vectors that are related, and you would like to display them side by side.
You might have a character vector and you would like to display it side by side with a Boolean vector indicating the positions of vowels in the character vector, so that you can verify visually if the result is correct:

In [113]:
text ← 'National Aeronautics and Space Administration'
vowels ← 0 1 0 1 1 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0

In [114]:
text,[0.5]vowels

N a t i o n a l A e r o n a u t i c s a n d S p a c e A d m i n i s t r a t i o n
0 1 0 1 1 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0

By looking at the above, you would be able to see more easily that your result missed the capital vowels, and you would then proceed to fixing your function.

This was a simple example of where laminating two vectors might prove useful, but this ad-hoc technique might also be useful in slightly more complex scenarios.
Suppose that we have four vectors containing information about certain products: their price, their current discount, their quantity in stock, and their availability (physical store, online, or both):

In [118]:
⎕← price ← 2 4 15 8 23
⎕← discount ← 0 0 25 25 35
⎕← stock ← 103 98 50 23 64
⎕← availability ← 'SSBOB'

2 4 15 8 23

0 0 25 25 35

103 98 50 23 64

SSBOB

The output shown above is not ideal, because each individual vector is displayed using its own natural format, and it is extremely difficult to connect the four related pieces of information to a specific product.
We can have these values displayed much better if we create a matrix.

To produce a matrix we will need to laminate two of the vectors and catenate the others (in rows or in columns).
The results are much easier to read:

In [119]:
price⍪discount⍪stock⍪[0.5]availability

2 4 15 8 23
 0 0 25 25 35
103 98 50 23 64
 S S B O B

In [120]:
price,discount,stock,[1.5]availability

2 0 103 S
 4 0 98 S
15 25 50 B
 8 25 23 O
23 35 64 B

In these examples, there is only one _laminate_, followed by as many _catenates_ as needed.

## Expand

### Basic Use

You remember that simple _compress_ uses a Boolean vector of 1s and 0s as a mask to include or exclude specific items of an array.

Simple _expand_ (specified by the `\` symbol) also uses a Boolean vector of 1s and 0s, but the 0s _insert_ new items into the array.
It is used as follows: `r ← pattern\argument`.

In this form, the Boolean vector left argument contains a 1 for each item of the right argument, and a 0 for each item to insert.
For example:

In [121]:
1 1 0 1 0 0 1 1 1\11 28 32 40 57 69

11 28 0 32 0 0 40 57 69

In [122]:
1 1 0 1 0 0 1 1 1\'Africa'

Af r ica

If the right argument is numeric, _expand_ inserts zeroes, and if it is a character vector, _expand_ inserts blanks as _fill items_.
For mixed or nested arrays, the concept of _fill item_ is more complex and, as mentioned before, will be explained in [a later section](./Nested-Arrays-(Continued).ipynb#Prototype-Fill-Item).

### Extended Definition

We can extend the behaviour of _extend_ to handle cases where the left argument is not a simple Boolean vector, but contains integers other than just 0s and 1s:

 - the amount of positive numbers in the left argument should match the length of the right argument;
 - for each positive item in the left argument, the corresponding item in the right argument is replicated as many times as is specified by that value;
 - each negative item in the left argument inserts an equivalent number of _fill items_ in the same position; and
 - zeroes in the left argument mean the same as `¯1`, and they each insert one fill item.

In the description above, we assumed that the right argument is a vector.
We will cover the fully generic case in a bit.

Because it is an extension of it, this new definition is fully compatible with the Boolean case we described before.

Here is an example:

In [124]:
1 1 0 3 ¯2 1 1 1\11 28 32 40 57 69

11 28 0 32 32 32 0 0 40 57 69

The first two items remain unchanged.
Then a zero inserts a zero in the result.
The next value is repeated 3 times, and the value `¯2` inserts 2 zeroes.
The last 3 items are unchanged.

The same thing can be done using a character vector:

In [125]:
1 1 0 3 ¯2 1 1 1\'expand'

ex ppp and

Because `0` and `¯1` have the same effect when used in the left argument of _expand_, we can obtain the same result with a different pattern:

In [126]:
1 1 ¯1 3 ¯2 1 1 1\'expand'

ex ppp and

Naturally, the function can work on any shape of array, in which case `\` acts on the last axis of the right argument, by default.

For example, if we take our `chemistry` matrix:

In [127]:
⎕← chemistry ← 3 5⍴'H2SO4CaCO3Fe2O3'

H2SO4
CaCO3
Fe2O3

We can insert one extra column by using _expand_ with a Boolean left argument:

In [128]:
1 1 0 1 1 1\chemistry

H2 SO4
Ca CO3
Fe 2O3

We can further specify the axis along which we want to act, to change the default:

In [129]:
1 0 1 0 1\[1]chemistry

H2SO4
 
CaCO3
 
Fe2O3

The axis specified can be the same as the default, making it redundant, but otherwise completely correct:

In [130]:
1 ¯3 1 1 1 0 3\[2]chemistry

H 2SO 444
C aCO 333
F e2O 333

With arrays of rank higher than 1, the description provided above remains the same, except that _extend_ no longer operates on single items of the right argument, but on its sub-arrays along the given axis (the last one, if left unspecified) instead.

_Expand_ can also be used on scalars; they are repeated as many times as necessary to fit the number of positive values in the left argument:

In [131]:
0 0 1 1 0 0 1 1\'A'

AA AA

The above is equivalent to

In [132]:
0 0 1 1 0 0 1 1\'AAAA'

AA AA

In [133]:
0 1 3 ¯2 1 1\71

0 71 71 71 71 0 0 71 71

### Expand Along First Axis

Like it was mentioned before, _expand_ works on the last dimension of an array.
To work on the first dimension, one can use the function `⍀` (<kbd>APL</kbd>+<kbd>.</kbd>):

In [134]:
1 1 ¯2 1⍀chemistry

H2SO4
CaCO3
 
 
Fe2O3

If one places an axis indication after the symbol `\` or `⍀`, the operation is processed according to the _axis_ operator, whichever of the two symbols is used.
For example:

 - `vec⍀[3]prod` and `vec\[3]prod` would be equivalent to `vec\prod`; and
 - `vec\[1]forecast` and `vec⍀[1]forecast` would be equivalent to `vec⍀forecast`.

## Reverse and Transpose

APL is also well endowed with functions which pivot data about an axis, and the axis is suggested by the shape of the symbol used.
The functions apply to both numeric and character data.
In the examples we are going to use a character matrix called `towns`:

In [144]:
⎕← towns ← 6 10⍴'Canberra  Paris     WashingtonMoscow    Martigues Mexico    '

Canberra 
Paris 
Washington
Moscow 
Martigues 
Mexico

The symbols `⌽` (<kbd>APL</kbd>+<kbd>Shift</kbd>+<kbd>5</kbd>) and `⊖` (<kbd>APL</kbd>+<kbd>Shift</kbd>+<kbd>7</kbd>) are used for two variants of the same function, which is called _reverse_.

The function `⍉` is called _transpose_ and its symbol is typed with <kbd>APL</kbd>+<kbd>Shift</kbd>+<kbd>6</kbd>.

Here are their effects:

In [145]:
(⊢towns) (⌽towns) (⊖towns) (⍉towns)

┌──────────┬──────────┬──────────┬──────┐
│Canberra │ arrebnaC│Mexico │CPWMMM│
│Paris │ siraP│Martigues │aaaoae│
│Washington│notgnihsaW│Moscow │nrssrx│
│Moscow │ wocsoM│Washington│bihcti│
│Martigues │ seugitraM│Paris │esioic│
│Mexico │ ocixeM│Canberra │r nwgo│
│ │ │ │r g u │
│ │ │ │a t e │
│ │ │ │ o s │
│ │ │ │ n │
└──────────┴──────────┴──────────┴──────┘

The symbols used (`⌽ ⊖ ⍉`) are self-describing, no effort is required to remember any of them because the position of the bar visually indicates which kind of transformation they stand for.

If you insert an axis specification after the symbols `⌽` or `⊖`, the operation is processed according to the _axis_ operator, whichever of the two symbols is used.
So, for example:

 - `⌽[1]matrix` and `⊖[1]matrix` are both equivalent to `⊖matrix`; and
 - `⌽[2]matrix` and `⊖[2]matrix` are both equivalent to `⌽matrix`.

### Caveats to be Aware Of

 - _Transpose_ has no effect on a vector, because it has only one axis:

In [146]:
⍉'I shall not move'

I shall not move

On a similar note,

 - The distracted APLer might expect `⊖` to do nothing on vectors, but that is _not_ what happens.
 
Visually, `⊖` hints at the fact that it "flips" its argument upside down, and a vector can't be flipped upside down because it is one-dimensional.
However, `⊖` is called _reverse **first**_, which means it always reverses the argument along its **first** dimension, even if that dimension is the _only_ dimension of the argument:

In [155]:
⊖'I shall not move...!?'

?!...evom ton llahs I

This confusion might arise from the visual similarity of the character vector above and

In [156]:
1 21⍴'I shall not move...!?'

I shall not move...!?

The latter is a 1-row matrix, which is why it _is_ left unchanged by `⊖`:

In [159]:
⊖1 21⍴'I shall not move...!?'

I shall not move...!?

 - _Transpose_ cannot be modified by an axis specifier, because it always operates on all of the dimensions of its argument.

 - _Transpose_ can be applied to arrays of any rank; let us try it with a 3D character array:

In [150]:
⎕← big ← names,[0.5]surnames

Alex
Anna
Mark
 
Gain
Nair
Tyte

In [151]:
⍉big

AG
AN
MT
 
la
na
ay
 
ei
ni
rt
 
xn
ar
ke

In [152]:
⍴big

2 3 4

In [153]:
⍴⍉big

4 3 2

You can see that `⍴⍉big` is equal to `⌽⍴big`, and the explanation of why this is the case will follow shortly.

All three primitives we just discussed also have (related) dyadic meanings, and those are introduced in two following sections.

## Rotate

The symbols `⌽` and `⊖`, when used dyadically, shift the items of the right argument in a circular manner.

The dyadic functions are called _rotate_.

### Rotate Vectors

Much like in the monadic case, the dyadic usage of `⌽` and `⊖` is identical when applied to vectors (we shall use `⌽` in our examples because of the visual clue that the symbol `⌽` gives with respect to the operation it performs).

Dyadic `⌽` (and `⊖`) expects an integer as the left argument, when the right argument is a vector, like so: `r ← n⌽vector`.

 - when `n` is positive, the first `n` items of `vector` are moved to the end. In other words, the vector is rotated to the left; and
 - when `n` is negative, the last `n` items of `vector` are moved to the beginning. In other words, the vector is rotated to the right:

In [164]:
7⌽'What did they do to my song?'

d they do to my song?What di

In the example above, the first 7 items of the vector (`'What di'`) have been moved to the back, whereas if we use a negative argument:

In [165]:
¯7⌽'What did they do to my song?'

y song?What did they do to m

the last 7 items of the vector (`'y song?'`) are moved to the front of the vector.

_Rotate_ can of course be applied to numeric vectors as well:

In [166]:
nums

56 66 19 37 44 20 18 23 68 70 82

In [167]:
3⌽nums

37 44 20 18 23 68 70 82 56 66 19

Do not confuse the following two expressions:

In [168]:
¯3⌽nums

68 70 82 56 66 19 37 44 20 18 23

In [169]:
-3⌽nums

¯37 ¯44 ¯20 ¯18 ¯23 ¯68 ¯70 ¯82 ¯56 ¯66 ¯19

The first one moves 3 items to the beginning, whilst the second expression moves the first 3 items to the end, and then changes the sign of the result (we saw something very similar when we talked about _take_).
It is all about being careful with the normal and high minus symbols!

### Rotate Higher-Rank Arrays

When applied to a matrix or higher-order array, `⌽` works on the last dimension, while `⊖` works on the first dimension.
This default behaviour can be overriden by an _axis_ specification.
To obtain a rotation along any other dimension, the _axis_ specification is mandatory.

_Rotate_ can be applied to any array, but we shall only demonstrate its application to matrices.

#### Uniform Rotation

In its simplest form, _rotate_ applies the same rotation to all the rows or columns of a matrix; let us see the result produced on a character matrix:

In [170]:
⎕← monMat ← 6 8⍴'January FebruaryMarch   April   May     June    '

January 
February
March 
April 
May 
June

In [171]:
2⌽monMat

nuary Ja
bruaryFe
rch Ma
ril Ap
y Ma
ne Ju

In [172]:
2⊖monMat

March 
April 
May 
June 
January 
February

In [173]:
¯2⊖monMat

May 
June 
January 
February
March 
April

#### Multiple Rotations

It is possible to apply a different rotation to each of the rows or to each of the columns.

In this case, the rotation is no longer indicated by a single value, but by a vector which specifies the amount by which each row or column will be moved.

In [174]:
chemistry

H2SO4
CaCO3
Fe2O3

In [175]:
¯1 0 2⌽chemistry

4H2SO
CaCO3
2O3Fe

Notice how `¯1` was used to rotate the first row, `0` was used to rotate the second row, and `2` was used to rotate the third row.

Using `⊖`, we can rotate columns:

In [176]:
monMat

January 
February
March 
April 
May 
June

In [177]:
1 0 2 ¯2 0 0 2 2⊖monMat

Far ar 
Mereua 
Aayuh 
Mpnrl 
Janc y 
Jubi ry

#### Application

_Rotate_ can provide very simple solutions to many tasks.
For example, let us count how many blanks appear at the end of each row of `monMat`:

In [178]:
+/' '=monMat

1 0 3 3 5 4

We can then use these values to move the blanks to the beginning of each row, thereby right-aligning the matrix:

In [180]:
(-+/' '=monMat)⌽monMat

January
February
 March
 April
 May
 June